In [95]:
import pandas as pd

In [343]:
residuals = pd.DataFrame({'area': ['pnos', 'vnp'], 'number': [3, 4], 'balances':[15,95]})


In [344]:
orders = pd.DataFrame({'area': ['pnos', 'vnp'], 'number': [3, 4], 'otgruzheno':[20,3]})

In [345]:
orders

,area,number,otgruzheno
0,pnos,3,20
1,vnp,4,3


In [346]:
20.0 - 19.1

0.8999999999999986

In [347]:
residuals

,area,number,balances
0,pnos,3,15
1,vnp,4,95


In [348]:
residuals = pd.merge(residuals, orders, how="left")
residuals.otgruzheno.fillna(0, inplace=True)


In [349]:
residuals

,area,number,balances,otgruzheno
0,pnos,3,15,20
1,vnp,4,95,3


In [350]:
residuals['cum_sum'] = residuals.groupby(['area', 'number']).otgruzheno.cumsum()
residuals = residuals.drop_duplicates(subset=['area','number'], keep='last')
residuals['new_balance'] = (residuals['balances'] - residuals['cum_sum']).clip(lower=0)

In [351]:
residuals

,area,number,balances,otgruzheno,cum_sum,new_balance
0,pnos,3,15,20,20,0
1,vnp,4,95,3,3,92


In [183]:
orders.balances - orders.requires_cum + orders.requires

0    20.0
1    95.0
2    25.0
3     5.0
4     0.0
dtype: float64

In [128]:
orders["ready"] = orders.requires.clip(upper=orders.balances - orders.requires_cum + orders.requires).clip(lower=0)
orders

,area,number,requires,balances,requires_cum,ready
0,pnos,3,40,20.0,40,20.0
1,vnp,4,70,95.0,70,70.0
2,vnp,4,20,95.0,90,20.0
3,vnp,4,10,95.0,100,5.0
4,vnp,1,25,0.0,25,0.0


In [110]:
residuals[(residuals['number']==3) & (residuals['area']=='pnos')]

,area,number,balances
0,pnos,3,20


In [117]:
def distr(number, area, requiers):
    if residuals[(residuals['number']==number) & (residuals['area']==area)].empty:
        print(type(number), type(requiers))
        print(number, requiers)
        print('empty')
        return 0
    elif requiers==0:
        return 0
    elif requiers>0:
        if residuals[(residuals['number']==number) & (residuals['area']==area)]['balances'].iloc[0]  >= requiers:    
            residuals.loc[(residuals['number']==number) & (residuals['area']==area), 'balances'] -= requiers
            return requiers
        else:
 
            result = residuals[(residuals['number']==number) & 
                               (residuals['area']==area)]['balances'].iloc[0]
            residuals.loc[(residuals['number']==number) & (residuals['area']==area), 'balances'] = 0
            
            return result

In [118]:
orders['ready'] = orders.apply(lambda x: distr(x['number'], x['area'], x['requires']), axis=1)

<class 'int'> <class 'int'>
1 25
empty


In [119]:
orders

,area,number,requires,ready
0,pnos,3,40,20
1,vnp,4,70,70
2,vnp,4,20,20
3,vnp,4,10,5
4,vnp,1,25,0


In [ ]:
def raspredelenie(ksss, neispolneno, area):
    
        
    #if (code_of_product not in req_3_factory_group[req_3_factory_group['area']==area]['number_product'].values) & (area not in req_3_factory_group[req_3_factory_group['number_product']==code_of_product]['area']):
    if fiz_ostatki[(fiz_ostatki['number_product']==ksss) & (fiz_ostatki['area']==area)].empty:
        return 0
       
    elif neispolneno==0:
        return 0
        
    elif neispolneno>0:
#         print('ksss: ' + str(ksss) + " " + str(area))
        if fiz_ostatki[(fiz_ostatki['number_product']==ksss) & 
                               (fiz_ostatki['area']==area)]['balances'].iloc[0]  >= neispolneno:    
            fiz_ostatki.loc[(fiz_ostatki['number_product']==ksss) & (fiz_ostatki['area']==area), 'balances'] -= neispolneno
#             print(neispolneno)
            return neispolneno
           
    
        else:
 
            result = fiz_ostatki[(fiz_ostatki['number_product']==ksss) & 
                               (fiz_ostatki['area']==area)]['balances'].iloc[0]
          
            fiz_ostatki.loc[(fiz_ostatki['number_product']==ksss) & (fiz_ostatki['area']==area), 'balances'] = 0
            
            return result

In [42]:
orders['дата производства'] = orders.apply(lambda x: data_shipment(x['Пункт отгрузки'], x['КССС мат'], x['осталось произвести, тн']), axis=1)

НАЧАЛО:
  Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
0            ВНП   1779443        2022-06-11                 8
required_to_make:  8
Объем больше
graphic1:    Пункт отгрузки  Код КССС Дата_производства  Объем_фасовки, т
0            ВНП   1779443        2022-06-11                 0


In [43]:
graphic

,Пункт отгрузки,Код КССС,Дата_производства,"Объем_фасовки, т"
0,ВНП,1779443,2022-06-11,0


In [44]:
orders

,КССС мат,Пункт отгрузки,"осталось произвести, тн",дата производства
0,1779443,ВНП,8,2022-06-11


In [712]:
graphic.loc[(graphic['Пункт отгрузки']=='ПНОС') & (graphic['Код КССС']==207465), 'Объем_фасовки, т']

Series([], Name: Объем_фасовки, т, dtype: int64)

In [713]:
graphic[(graphic['Пункт отгрузки']=="ПНОС") & (graphic['Код КССС']==20746)].empty

True

In [714]:
##делать колонку с тем, сколько не хватает еще бессмысленно, так как продукт может далее распределиться по другим заказам, 
#для которых хватает и тогда число не будет иметь значение

In [715]:
# проверить если не накопилась дважды или трижды

In [129]:
data = {'col_0': [9, -3, 0, -1, 5], 'col_1': [-2, -7, 6, 8, -5]}
>>> df = pd.DataFrame(data)
>>> df

,col_0,col_1
0,9,-2
1,-3,-7
2,0,6
3,-1,8
4,5,-5


In [177]:
t = pd.Series([-7,-100,-50,-20])
>>> t

0     -7
1   -100
2    -50
3    -20
dtype: int64

In [178]:
df.clip(t, t + 4, axis=0)

,col_0,col_1
0,-3.0,-3.0
1,-96.0,-96.0
2,-46.0,-46.0
3,-16.0,-16.0
4,NaN,NaN


In [179]:
t+4

0    -3
1   -96
2   -46
3   -16
dtype: int64